In [ ]:
!pip install pandas matplotlib seaborn sqlalchemy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# --- Thiết lập môi trường ---
# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối từ biến môi trường
db_user = os.getenv('MYSQL_USER')
db_password = os.getenv('MYSQL_PASSWORD')
db_host = os.getenv('MYSQL_HOST')
db_port = os.getenv('MYSQL_PORT')
db_name = os.getenv('MYSQL_DATABASE')

# --- Tạo kết nối tới Database ---
# Chuỗi kết nối (connection string)
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

try:
    engine = create_engine(connection_string)
    print("Kết nối tới database thành công!")
except Exception as e:
    print(f"Lỗi kết nối database: {e}")

# --- Thiết lập style cho biểu đồ ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

In [ ]:
# --- Phân tích 1: Doanh thu theo bang ---

# 1. Viết câu lệnh SQL
query_revenue_by_state = """
SELECT
    c.customer_state,
    SUM(oi.price + oi.freight_value) AS total_revenue
FROM
    orders o
JOIN
    customers c ON o.customer_id = c.customer_id
JOIN
    order_items oi ON o.order_id = oi.order_id
GROUP BY
    c.customer_state
ORDER BY
    total_revenue DESC
LIMIT 10;
"""

# 2. Thực thi và lấy dữ liệu
df_revenue_state = pd.read_sql(query_revenue_by_state, engine)
print("Top 10 bang theo doanh thu:")
display(df_revenue_state)

# 3. Trực quan hóa
sns.barplot(
    data=df_revenue_state,
    x='total_revenue',
    y='customer_state',
    palette='viridis'
)
plt.title('Top 10 Bang có Doanh thu Cao nhất')
plt.xlabel('Tổng Doanh thu (triệu R$)')
plt.ylabel('Bang')
plt.show()

In [ ]:
# --- Phân tích 2: Thời gian giao hàng và Review Score ---

# 1. Viết câu lệnh SQL
# DATEDIFF dùng để tính số ngày chênh lệch
query_delivery_time_review = """
SELECT
    r.review_score,
    AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)) AS avg_delivery_days
FROM
    orders o
JOIN
    order_reviews r ON o.order_id = r.order_id
WHERE
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
GROUP BY
    r.review_score
ORDER BY
    r.review_score ASC;
"""

# 2. Thực thi và lấy dữ liệu
df_delivery_review = pd.read_sql(query_delivery_time_review, engine)
print("Thời gian giao hàng trung bình theo từng Review Score:")
display(df_delivery_review)

# 3. Trực quan hóa
sns.barplot(
    data=df_delivery_review,
    x='review_score',
    y='avg_delivery_days',
    palette='coolwarm'
)
plt.title('Thời gian Giao hàng Trung bình và Điểm Đánh giá')
plt.xlabel('Điểm Đánh giá (Review Score)')
plt.ylabel('Số ngày Giao hàng Trung bình')
plt.show()

In [ ]:
# --- Phân tích 3: Top danh mục sản phẩm theo doanh thu ---

# 1. Viết câu lệnh SQL
query_top_categories = """
SELECT
    p.product_category_name,
    SUM(oi.price) AS total_revenue
FROM
    order_items oi
JOIN
    products p ON oi.product_id = p.product_id
GROUP BY
    p.product_category_name
ORDER BY
    total_revenue DESC
LIMIT 10;
"""

# 2. Thực thi và lấy dữ liệu
df_top_categories = pd.read_sql(query_top_categories, engine)
print("Top 10 danh mục sản phẩm theo doanh thu:")
display(df_top_categories)

# 3. Trực quan hóa
sns.barplot(
    data=df_top_categories,
    x='total_revenue',
    y='product_category_name',
    palette='plasma'
)
plt.title('Top 10 Danh mục Sản phẩm có Doanh thu Cao nhất')
plt.xlabel('Tổng Doanh thu')
plt.ylabel('Danh mục Sản phẩm')
plt.show()